In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
from gurobipy import *

In [2]:
import pandas as pd

In [3]:
import numpy as np

In [4]:
def PercentageOptimalityGap(optimal_value, alg_value):
    return (optimal_value - alg_value) / optimal_value

In [5]:
def min_info(arr):
    if(len(arr) == 0):
        return
    min_index = 0
    min_value = arr[min_index]
    for i in range(len(arr)):
        if(arr[i] < min_value):
            min_index = i
            min_value = arr[min_index]
    return min_index, min_value

In [6]:
def int_to_string(number):
    if(number < 10):
        return '0' + str(number)
    else:
        return str(number)

In [7]:
'''
IP formulation
Input: m,n,k,b,p
Output: optimal value
'''
def IP_answer(M,N,K,B,P):
    p3_b = Model("p3_b")
    
    # a_jm
    a = {}

    for j in range(N):
        for m in range(M):
            a[j,m] = p3_b.addVar(lb = 0, vtype = GRB.BINARY, name = "a_" + str(j+1) + "_" + str(m+1))

    # w 
    w = p3_b.addVar(lb = 0, vtype= GRB.CONTINUOUS, name = "w")
    
    # obj
    p3_b.setObjective(w, GRB.MAXIMIZE)
    
    #1
    p3_b.addConstrs(((quicksum(a[j,m] for m in range(M)) <= 1) for j in range(N)), "1 job asigned to 1 machine" )

    #2
    p3_b.addConstrs(((quicksum(a[j,m]*P[j] for j in range(N)) <= K) for m in range(M)), "not exceed capacity")

    #3
    p3_b.addConstrs(((quicksum(B[j]*a[j,m] for j in range(N)) >= w) for m in range(M)), "w is the min")

    #Optimize
    p3_b.optimize()
    
    # Return Optimal Value
    return p3_b.objVal

In [8]:
'''
HBF formulation
Input: m,n,k,b,p
Output: optimal value
'''
def HBF_answer(m,n,k,b,p):
    # Init accumulated processing time and benefit of each machine
    P = [0 for i in range(m)]
    B = [0 for i in range(m)]
    
    # Sorting the job
    for i in range(n):
        for j in range(0, n-i-1):
            if b[j+1] > b[j]:
                b[j], b[j+1] = b[j+1], b[j]
                p[j], p[j+1] = p[j+1], p[j]
            
    # Main Heuristic Part
    for i in range(n):
#         temp_accumulated_benefit = B
        temp_accumulated_benefit = {}
        for j in range(m):
            temp_accumulated_benefit[j] = B[j]
        for j in range(m):
            min_value = min(temp_accumulated_benefit)
            min_index =  min(temp_accumulated_benefit, key=temp_accumulated_benefit.get)
            if(k - P[min_index] - p[i] >= 0):
                P[min_index] += p[i]
                B[min_index] += b[i]
                break
            else:
                temp_accumulated_benefit.pop(min_index, None)
                
    min_index, min_benefit = min_info(B)
    return min_benefit

In [9]:
'''
HRF formulation
Input: m,n,k,b,p
Output: optimal value
'''
def HRF_answer(m,n,k,b,p):
    # Init accumulated processing time and benefit of each machine
    P = [0 for i in range(m)]
    B = [0 for i in range(m)]
    
    # Sorting the job
    for i in range(n):
        for j in range(0, n-i-1):
            if (b[j+1]/p[j+1]) > (b[j]/p[j]):
                b[j], b[j+1] = b[j+1], b[j]
                p[j], p[j+1] = p[j+1], p[j]
            
    # Main Heuristic Part
    for i in range(n):
#         temp_accumulated_benefit = B
        temp_accumulated_benefit = {}
        for j in range(m):
            temp_accumulated_benefit[j] = B[j]
        for j in range(m):
            min_value = min(temp_accumulated_benefit)
            min_index =  min(temp_accumulated_benefit, key=temp_accumulated_benefit.get)
            if(k - P[min_index] - p[i] >= 0):
                P[min_index] += p[i]
                B[min_index] += b[i]
                break
            else:
                temp_accumulated_benefit.pop(min_index, None)
                
    min_index, min_benefit = min_info(B)
    return min_benefit

In [10]:
'''
My formulation
Input: m,n,k,b,p
Output: optimal value
'''
def My(m,n,k,b,p):
    # Init accumulated processing time and benefit of each machine
    P = [0 for i in range(m)]
    B = [0 for i in range(m)]
    
    # Sorting the job
    for i in range(n):
        for j in range(0, n-i-1):
            if (b[j+1]/p[j+1]) > (b[j]/p[j]):
                b[j], b[j+1] = b[j+1], b[j]
                p[j], p[j+1] = p[j+1], p[j]
            elif((b[j+1]/p[j+1]) == (b[j]/p[j]) and b[j+1] < b[j]):
                b[j], b[j+1] = b[j+1], b[j]
                p[j], p[j+1] = p[j+1], p[j]
            
    # Main Heuristic Part
    for i in range(n):
#         temp_accumulated_benefit = B
        temp_accumulated_benefit = {}
        for j in range(m):
            temp_accumulated_benefit[j] = B[j]
        for j in range(m):
            min_value = min(temp_accumulated_benefit)
            min_index =  min(temp_accumulated_benefit, key=temp_accumulated_benefit.get)
            if(k - P[min_index] - p[i] >= 0):
                P[min_index] += p[i]
                B[min_index] += b[i]
                break
            else:
                temp_accumulated_benefit.pop(min_index, None)
                
    min_index, min_benefit = min_info(B)
    return min_benefit

# 4-a

In [11]:
# Create Dataframe
data = {
    'IP': [0 for i in range(20)],
    'HBF': [0 for i in range(20)],
    'HRF': [0 for i in range(20)],
    'My':[0 for i in range(20)]
}
df_answer = pd.DataFrame(data)
df_answer

,IP,HBF,HRF,My
0,0,0,0,0
1,0,0,0,0
2,0,0,0,0
3,0,0,0,0
4,0,0,0,0
5,0,0,0,0
6,0,0,0,0
7,0,0,0,0
8,0,0,0,0
9,0,0,0,0


In [12]:
for i in range(20):
    filename = 'data/in' + int_to_string(i+1) + '.txt'
    with open(filename,'r') as f:
        m,n,k = f.readline().split()
        m = int(m)
        n = int(n)
        k = int(k)
        b = [int(i) for i in f.readline().split()]
        p = [int(i) for i in f.readline().split()]
    
        df_answer['IP'][i] = IP_answer(m,n,k,b,p)
        df_answer['HBF'][i] = HBF_answer(m,n,k,b,p)
        df_answer['HRF'][i] = HRF_answer(m,n,k,b,p)
        df_answer['My'][i] = My(m, n, k, b, p)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-03-28
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 51 rows, 136 columns and 408 nonzeros
Model fingerprint: 0xac92817b
Variable types: 1 continuous, 135 integer (135 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective -0.0000000
Presolve time: 0.02s
Presolved: 51 rows, 136 columns, 408 nonzeros
Variable types: 0 continuous, 136 integer (135 binary)
Found heuristic solution: objective 1.0000000

Root relaxation: objective 1.467000e+02, 84 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  146.

C:\Users\user\AppData\Local\Temp\ipykernel_25824\3375148794.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_answer['HBF'][i] = HBF_answer(m,n,k,b,p)
C:\Users\user\AppData\Local\Temp\ipykernel_25824\3375148794.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_answer['HRF'][i] = HRF_answer(m,n,k,b,p)
C:\Users\user\AppData\Local\Temp\ipykernel_25824\3375148794.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_answer['M

H    0     0                      60.2500000   76.06667  26.3%     -    0s
H    0     0                      64.2500000   76.06667  18.4%     -    0s
H    0     0                      66.0000000   76.06667  15.3%     -    0s
H    0     0                      67.0000000   76.06667  13.5%     -    0s
H    0     0                      68.2500000   76.06667  11.5%     -    0s
     0     0   75.84000    0   17   68.25000   75.84000  11.1%     -    0s
H    0     0                      69.0000000   75.84000  9.91%     -    0s
H    0     0                      71.0000000   75.84000  6.82%     -    0s
     0     0   75.75000    0   15   71.00000   75.75000  6.69%     -    0s
H    0     0                      72.0000000   75.75000  5.21%     -    0s
     0     0   75.75000    0   26   72.00000   75.75000  5.21%     -    0s
     0     0   75.75000    0   19   72.00000   75.75000  5.21%     -    0s
H    0     0                      73.0000000   75.75000  3.77%     -    0s
     0     0   75.75000  

In [13]:
df_answer

,IP,HBF,HRF,My
0,146.0,144,145,144
1,95.0,89,92,92
2,54.0,50,51,51
3,144.0,136,140,143
4,59.0,49,55,55
5,115.0,108,112,114
6,83.0,76,80,78
7,76.0,70,71,72
8,85.0,80,80,80
9,138.0,137,136,137


In [14]:
for index, row in df_answer.iterrows():
    if(row['IP'] < row['HBF'] or row['IP'] < row['HRF']):
        print("error")
print("pass")

pass


## Calculate Percentage Optimality Gap

In [15]:
gap_HBF_and_IP = []
gap_HRF_and_IP = []
gap_My_and_IP = []
for i in range(20):
    gap_HBF_and_IP.append(PercentageOptimalityGap(df_answer['IP'][i], df_answer['HBF'][i]))
    gap_HRF_and_IP.append(PercentageOptimalityGap(df_answer['IP'][i], df_answer['HRF'][i]))
    gap_My_and_IP.append(PercentageOptimalityGap(df_answer['IP'][i], df_answer['My'][i]))

In [16]:
df_answer['HBF Gap'] = gap_HBF_and_IP
df_answer['HRF Gap'] = gap_HRF_and_IP
df_answer['My Gap'] = gap_My_and_IP

In [17]:
df_answer

,IP,HBF,HRF,My,HBF Gap,HRF Gap,My Gap
0,146.0,144,145,144,0.013699,0.006849,0.013699
1,95.0,89,92,92,0.063158,0.031579,0.031579
2,54.0,50,51,51,0.074074,0.055556,0.055556
3,144.0,136,140,143,0.055556,0.027778,0.006944
4,59.0,49,55,55,0.169492,0.067797,0.067797
5,115.0,108,112,114,0.060870,0.026087,0.008696
6,83.0,76,80,78,0.084337,0.036145,0.060241
7,76.0,70,71,72,0.078947,0.065789,0.052632
8,85.0,80,80,80,0.058824,0.058824,0.058824
9,138.0,137,136,137,0.007246,0.014493,0.007246


In [18]:
print("Average percentage optimality gap of HBF  = ", np.mean(df_answer['HBF Gap']))
print("Average percentage optimality gap of HRF  = ", np.mean(df_answer['HRF Gap']))
print("Average percentage optimality gap of My  = ", np.mean(df_answer['My Gap']))


Average percentage optimality gap of HBF  =  0.0709581596325218
Average percentage optimality gap of HRF  =  0.050576676743551596
Average percentage optimality gap of My  =  0.04981516129637063


# 4-b

In [20]:
print("average:")
print("IP:",  df_answer['IP'].mean())
print("HBF:", df_answer['HBF'].mean())
print("HRF:", df_answer['HRF'].mean())
print("My:", df_answer['My'].mean())


average:
IP: 90.25000000350438
HBF: 84.35
HRF: 86.4
My: 86.5
